<h1> N-Able - Download All Scripts </h1>

# Import Modules and Define Variables

In [ ]:
# file manipulation
import os
import csv

# API and Web Requests
import requests
import urllib3 # make url requests
import shutil # manage packages

# web scraping
from selenium import webdriver


#data conditioning
import pandas as pd
import re
import datetime as dt

In [ ]:
 # add current timestamp to filename for reference
current_time = (dt.datetime.utcnow().strftime('%Y_%m_%d_%H%M%S'))

# git repo folder
git_folder = 'd:/git/example_infrastructure_data_dev'

# dictionary Directory
dictionary_dir = 'd:/git/example_infrastructure_data_dev/dictionaries'

# source dir for nable exported data
source_folder = 'd:/project_docs/abc_nable_migration/abc_nable_exports/patch_management'

# export folder will contain all csv exported DataFrames for Ticket Creation
export_folder = 'd:/exports'

In [ ]:
# import configparser for env secrets
from configparser import ConfigParser

config = ConfigParser()
config.read(f'{git_folder}/config/env.ini')
import requests
from requests.structures import CaseInsensitiveDict

## Create Firefox Driver

In [ ]:
from selenium.webdriver.firefox.options import Options

mozilla_driver = 'd:/web_drivers/mozilla/geckodriver-v0.31.0-win64/geckodriver.exe'

options = Options()
options.set_preference("browser.download.folderList", 2)
options.set_preference("browser.download.manager.showWhenStarting", False)
options.set_preference("browser.download.dir", f"{export_folder}/script_exports/")
options.set_preference("browser.helperApps.neverAsk.saveToDisk", "application/x-gzip")

f_driver = webdriver.Firefox(executable_path=mozilla_driver,options=options)

In [ ]:
baseUrl = 'https://ncod405.n-able.com/'
auth_url = f'https://ncod405.n-able.com/so/abc'

links = []

In [ ]:
f_driver.get(auth_url)

In [ ]:
f_driver.find_element_by_xpath("//div[@class='pageTitleBarTextDiv']").text

## Request Cookies to Create New Header

In [ ]:
def get_cookies():
    cookies = {}
    selenium_cookies = f_driver.get_cookies()
    for cookie in selenium_cookies:
        cookies[cookie['name']] = cookie['value']
    return cookies

In [ ]:
cookies = get_cookies()

In [ ]:
cookies

In [ ]:
cookie = str(re.sub(r'[\'\{\}]',"",str(cookies))).replace(": ","=").replace(",",";")
print(cookie)

In [ ]:
headers = f_driver.execute_script("var req = new XMLHttpRequest();req.open('GET', document.location, false);req.send(null);return req.getAllResponseHeaders()")

headers_list = headers.splitlines()

In [ ]:
header_string = f"""Host: ncod405.n-able.com
User-Agent: Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:105.0) Gecko/20100101 Firefox/105.0
Accept: text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,*/*;q=0.8
Accept-Language: en-US,en;q=0.5
Accept-Encoding: gzip, deflate, br
Connection: keep-alive
Referer: https://ncod405.n-able.com/IndexAction.action
Cookie: {cookie}
Upgrade-Insecure-Requests: 1
Sec-Fetch-Dest: document
Sec-Fetch-Mode: navigate
Sec-Fetch-Site: same-origin
Sec-Fetch-User: ?1"""

In [ ]:
headers = {}

for header in re.findall(r'([^\n]+)\n',header_string):
    element_dict = {}
    result = re.match(r'([^\:]+)\:(.*)',header)
    k = (result.group(1)).lstrip().rstrip()
    v = (result.group(2)).lstrip().rstrip()
    element_dict[k] = v
    headers.update(element_dict)
print(headers)

# Pull all File Download Information from Script/ Software Repository

In [ ]:
column_names_list = ['customerName',
                     'name',
                     'description',
                     'size',
                     'status',
                     'uploadDate',
                     'fileName']

In [ ]:
def file_info_td(row):
    i = 0
    file_download_dict = {}
    file_download_dict['downloadNumber'] = row['downloadNumber']
    file_download_dict['downloadId'] = row['downloadId']
    # print('*'*100)
    # print(f"\nfinding info for {row['downloadId']}:\n")
    table_row = f_driver.find_element_by_xpath(f"//div[@id='{row['downloadId']}']")
    for td in table_row.find_elements_by_xpath(f'.//td'):
        item_dict = {}
        if td.text:
            if i < len(column_names_list):
                # print(f"assigning ['{column_names_list[i]}'] to {td.text}")
                item_dict[column_names_list[i]] = str(td.text).rstrip().lstrip()
                file_download_dict.update(item_dict)
                i = i+1
            else:
                download_url = f"https://ncod405.n-able.com/downloadFileServlet.download?relativePathToFile=%2Fdownload%2Frepository%2F{file_download_dict['downloadNumber']}%2F{str(file_download_dict['fileName']).replace(' ','+')}"
                # print(f"assigning ['downloadUrl'] to {download_url}")
                file_download_dict['downloadUrl'] = download_url
                file_download_list.append(file_download_dict)
                file_download_dict = {}

    print("\n")

In [ ]:
def download_file(row):
    # print(f"attempting to download {row['fileName']}")
    try:
        with requests.get(row['downloadUrl'], stream=True, allow_redirects=True, headers=headers) as r:
            r.raise_for_status()
            with open(f"{export_folder}/script_exports/{row['fileName']}", 'wb') as f:
                for chunk in r.iter_content(chunk_size=8192):
                    # If you have chunk encoded response uncomment if
                    # and set chunk_size parameter to None.
                    #if chunk:
                    f.write(chunk)

    except:
        print(f"error with {row['fileName']}")
        with open(f"{export_folder}.csv",'w+') as e:
            writer = csv.writer(e)
            writer.writerow({'fileName':row['fileName'], 'downloadUrl':row['downloadUrl']})

In [ ]:
software_js_table = f_driver.find_element_by_xpath('/html/body/div[1]/div[2]/div[1]/div[2]/div[2]/div')


file_download_id_list = []

for div in software_js_table.find_elements_by_xpath('//div[@id]'):
    file_download_id_dict = {}
    result = re.search(r'fileRepositoryGrid\-row\-(\d+)',div.get_property('id'))
    if result:
        file_download_id_dict['downloadNumber'] = str(result.group(1)).rstrip().lstrip()
        file_download_id_dict['downloadId'] = str(div.get_property('id')).rstrip().lstrip()
        file_download_id_list.append(file_download_id_dict)


df_file_downloads = pd.DataFrame(file_download_id_list)

file_download_list = []

for index, row in df_file_downloads.iterrows():
    file_info_td(row)

df_file_downloads = pd.DataFrame(file_download_list)

for index, row in df_file_downloads.iterrows():
    download_file(row)